In [1]:
# periodic

import numpy as np
import matplotlib.pyplot as plt
import sys
import time
import os




2024-10-30 01:03:44.148392: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-30 01:03:44.148417: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-30 01:03:44.149537: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 01:03:44.156357: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-30 01:03:47.210727: W tensorflow/compiler/tf2

In [2]:
cwd = os.getcwd()   # the current directory
print(cwd)



$u_0(x) = \frac{\sqrt{2}}{cosh(x-x_0)} exp(i\frac{c(x-x_0)}{2})$

In [6]:
n_sample = 500
example_time_steps = 6
n_time_steps = 2000
n_time_skip = 10


Nx = 256
N1 = Nx

l = 0.11
L = 2*np.pi/l

# c = .1
# T = 1.23
# K = 30


dx = L/Nx
dt = 0.01  # cfl = c*(dt/dx) ,c = sqrt(k(w)), k_max = 0.14246

x = np.arange(Nx)*dx-L/2
NNc = 11


# c_all = np.random.uniform(0.8,1.2,size = n_sample)
# x0_all = np.random.uniform(-5.0,5.0,size = n_sample)   
np.random.seed(123)
eps_all = np.random.uniform(0.9,1.1,size = n_sample)    #eps_all, If the memory is insufficient, can generate it in segments  
# eps_all = np.array([1]*n_sample)
# def init(s):
#     if s <= 1:
#         return 1.-1.5*s**2.+0.75*s**3.
#     elif 1<s<2:

#         return 0.25*(2.-s)**3.
#     return 0.


print(f'final time is {dt*n_time_steps}')
print(f'simulation time step is {dt}')
print(f'flow map time step is {dt*n_time_skip}')


Z = np.array([])

# loop for sample


# vinit = np.vectorize(init)

# x0 = 0.5 + np.random.uniform(-0.2,0.2,size=n_sample)
# a0 = np.random.uniform(7.,9.,size=n_sample)
# w_all = np.random.uniform(1,2,size=n_sample)

z_list = []
p_list = []
q_list = []

c = 1
x0 = 0
for eps in eps_all:
    q = np.sqrt(2)/np.cosh(x-x0)*np.sin(c*(x-x0)/2)
    p = np.sqrt(2)/np.cosh(x-x0)*np.cos(c*(x-x0)/2)
    z = np.concatenate([p, q], axis=0)

    z_list.append(z)
    p_list.append(p)
    q_list.append(q)

z = np.array(z_list) # convert list to numpy ndarray
p = np.array(p_list) # convert list to numpy ndarray
q = np.array(q_list) # convert list to numpy ndarray

# Z = z.copy()[np.newaxis,...]
Z_list = []
Z_list.append(z)

#print(p.shape)

final time is 20.0
simulation time step is 0.01
flow map time step is 0.1


In [12]:
from scipy.optimize import fsolve

In [13]:
def solve_p_half(p_n, q_n, eps_all, dt):
    p_half = np.zeros_like(p_n)

    for i in range(len(p_n)):
        def F(p_half_single):
            return nonlinear_step1(eps_all[i], p_half_single.reshape(1, -1), 
                                   p_n[i:i+1], q_n[i:i+1],dt).flatten()
        p_half[i] = fsolve(F, p_n[i], xtol=1e-12, maxfev=1000)

    return p_half

def nonlinear_step1(eps, p_half, p_n, q_n, dt):
    return (p_half - p_n + 
            (dt/2) * (periodic_laplacian(q_n) +
                      eps * (p_half**2 + q_n**2) * q_n))

def periodic_laplacian(y):
    return (np.roll(y, -1, axis=1) - 2 * y + np.roll(y, 1, axis=1)) / (dx**2)

In [39]:
def u(x,t,eps):
    
    # x0 = 0
    # c = 1
    amp = np.sqrt(2/eps) * 1./np.cosh(x-x0-c*t)
     # 计算相位
    phase = c*x/2 - (c**2/4-1)*t
    real_part = amp * np.cos(phase)
    imag_part = amp * np.sin(phase)
    return real_part, imag_part

In [14]:
def solve_q_new(q_n, p_half, eps_all, dt):
    q_new = np.zeros_like(q_n)
    
    for i in range(len(q_n)):
        def F(q_new_single):
            # 将q_new_single重塑为2D数组
            q_new_reshaped = q_new_single.reshape(1, -1)
            
            # 计算拉普拉斯项
            p_xx = periodic_laplacian(p_half[i:i+1])
            
            # 计算两个非线性项
            nonlinear_term1 = eps_all[i] * ((q_n[i:i+1])**2 + (p_half[i:i+1])**2) * p_half[i:i+1]
            nonlinear_term2 = eps_all[i] * ((q_new_reshaped)**2 + (p_half[i:i+1])**2) * p_half[i:i+1]
            
            # 构造残差方程
            residual = (q_new_reshaped - q_n[i:i+1] - 
                       (dt/2) * (p_xx + nonlinear_term1) -
                       (dt/2) * (p_xx + nonlinear_term2))
            
            return residual.flatten()
        
        # 使用fsolve求解
        q_new[i] = fsolve(F, q_n[i], xtol=1e-12, maxfev=1000)
    
    return q_new

In [14]:
q.shape

(5, 256)

### batch

$q_t = p_{xx}+\epsilon (q^2+p^2)p, p_t= -q_{xx}-\epsilon(q^2+p^2)q$

In [15]:
for i in range(1, 1+n_time_steps):


    #step 1: obtain p^{n+1/2}
    # p_init = p + dt/4*(-periodic_laplacian(q) - eps_all * (p**2 + q**2) * q)
    
    p_half = solve_p_half(p, q, eps_all, dt)
    # Step 2: Update q^{n+1}
    # p_xx = periodic_laplacian(p_half)
    # nonlinear_term = eps_all * (p_half**2 + q**2) * p_half
    # q = q + dt * (p_xx + nonlinear_term)
    q = solve_q_new(q, p_half, eps_all, dt)
    
    # Step 3: Update p^{n+1}
    q_xx = periodic_laplacian(q)
    nonlinear_term = eps_all * (p_half**2 + q**2) * q
    p = p_half + (dt/2) * (-q_xx - nonlinear_term)
    z = np.concatenate([p, q], axis=1)

    Z_list.append(z)

Z = np.array(Z_list)

print(Z.shape)



# # Z = Z[::n_time_skip,:,:]
# Z = Z[::n_time_skip]







# the shape of input training data is (n_time, n_sample, n_x) or (n_time * n_sample, n_x)



(2001, 100, 512)


In [18]:
Z = np.array(Z_list)
Z.shape

(2001, 100, 512)

In [47]:
n_time_steps1 = Z.shape[0]


output_list = []
for shift in range(1, example_time_steps+1):
# output time series, starting from each single time step

    output_slice = Z[shift: n_time_steps1 - example_time_steps + shift]
# merge time and sample dimension as required by training
    n_time, _, n_x = output_slice.shape
    output_slice = np.reshape(output_slice, [n_sample * n_time, n_x])
    output_list.append(output_slice)

labels = np.stack(output_list, axis=1)

features = Z[:-example_time_steps,:,:]
features = features.reshape([n_time*n_sample,N1*2])

In [ ]:
data_features = f"Schrodinger_features_Nx_{N1}_sample_{n_sample}_timesteps_{n_time}_dt_{dt*2*n_time_skip}"
data_labels = f"Schrodinger_labels_Nx_{N1}_sample_{n_sample}_timesteps_{n_time}_dt_{dt*2*n_time_skip}"
# ensure the data directory exists
data_dir = os.path.join(cwd, 'data')
os.makedirs(data_dir, exist_ok=True)  # create the data directory if it doesn't exist

features_file = os.path.join(cwd, "data", data_features + ".npy")
labels_file = os.path.join(cwd, "data", data_labels + ".npy")

print(data_features)



Schrodinger_features_Nx_256_sample_500_timesteps_95_dt_0.2


In [55]:
eps_all1 = np.concatenate([eps_all]*95,axis=1)
eps_all1 = eps_all1.T.reshape(-1,1)

(500, 95)

In [17]:
if not os.path.exists(features_file):
    os.makedirs(features_file)
if not os.path.exists(labels_file):
    os.makedirs(labels_file)

In [58]:
features1 = np.concatenate([features,eps_all1],axis=1)


(47500, 513)

In [62]:
target_file = np.save(data_features + ".npy",features1)
target_file = np.save(data_labels + ".npy",labels)